# Reading Files and Split Apply Combine

This lesson focuses on reviewing our basics with pandas and extending them to more advanced munging and cleaning.  Specifically, we will discuss how to load data files, work with missing values, use split-apply-combine, use string methods, and work with string and datetime objects.  By the end of this lesson you should feel confident doing basic exploratory data analysis using `pandas`. 

**OBJECTIVES**

- Read local files in as `DataFrame` objects
- Drop missing values
- Replace missing values
- Impute missing values
- Use `.groupby` 
- Use built in `.dt` methods
- Convert columns to `pd.datetime` datatype
- Work with `datetime` objects in pandas.


## Reading Local Files

To read in a local file, we need to pay close attention to our *working directory*.  This means the current location of your work enviornment with respect to your larger computer filesystem.  To find your working directory you can use the `os` library or if your system executes UNIX commands these can be used.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [2]:
#pip install seaborn

In [3]:
#check working directory
os.getcwd()

'/Users/jacobkoehler/Desktop/fall_23/NYU-Bootcamp/new_bootbook'

In [4]:
#list all files in directory
os.listdir()

['intro_to_numpy.ipynb',
 'homework_I.ipynb',
 'intro_to_plotting.ipynb',
 'homework_III.zip',
 'gdp.xlsx',
 'requirements.txt',
 'images',
 'homework_III.ipynb',
 'dates_intro_to_plotting.ipynb',
 'python_fundamentals_one.ipynb',
 'intro_to_pandas.ipynb',
 '__MACOSX',
 '_toc.yml',
 'logo.png',
 'data_apis.ipynb',
 '_build',
 '_config.yml',
 'Pandas_Cheat_Sheet.pdf',
 'python_fundamentals_two.ipynb',
 '.ipynb_checkpoints',
 'syllabus.ipynb',
 '.git',
 'plotting_seaborn.ipynb',
 'data',
 'gdp.csv',
 'pandas_II.ipynb']

In [5]:
#what's in the data folder?
os.listdir('data')

['Ames_Housing_Sales.csv',
 'diamonds.csv',
 'gapminder_all.csv',
 'LasVegasTripAdvisorReviews-Dataset.csv',
 'ufo.csv',
 'salesdaily.csv',
 'movie_ratings.tsv',
 'churn_missing.csv',
 'NBA_players_2015.csv',
 'spotify.csv',
 'cars.csv',
 'cell_phone_churn.csv',
 'rollingsales_manhattan.csv',
 'AAPL.csv']

In [6]:
#what is the path to ufo.csv?
ufo_path = 'data/ufo.csv'

##### `read_csv`

Now, using the path to the `ufo.csv` file, you can create a DataFrame by passing this filepath to the `read_csv` function.

In [7]:
#read in ufo data
ufo = pd.read_csv(ufo_path)

In [8]:
# look at first 2 rows
ufo.head(2)

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00


In [9]:
# high level information
ufo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80543 entries, 0 to 80542
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   City             80492 non-null  object
 1   Colors Reported  17034 non-null  object
 2   Shape Reported   72141 non-null  object
 3   State            80543 non-null  object
 4   Time             80543 non-null  object
dtypes: object(5)
memory usage: 3.1+ MB


In [10]:
# numerical summaries
ufo.describe()

,City,Colors Reported,Shape Reported,State,Time
count,80492,17034,72141,80543,80543
unique,13503,31,27,52,68901
top,Seattle,ORANGE,LIGHT,CA,7/4/2014 22:00
freq,646,5216,16332,10743,45


In [11]:
# categorical summaries
ufo.describe(include = 'object')

,City,Colors Reported,Shape Reported,State,Time
count,80492,17034,72141,80543,80543
unique,13503,31,27,52,68901
top,Seattle,ORANGE,LIGHT,CA,7/4/2014 22:00
freq,646,5216,16332,10743,45


In [12]:
# all summaries
ufo.describe(include = 'all')

,City,Colors Reported,Shape Reported,State,Time
count,80492,17034,72141,80543,80543
unique,13503,31,27,52,68901
top,Seattle,ORANGE,LIGHT,CA,7/4/2014 22:00
freq,646,5216,16332,10743,45


In [13]:
# tips = sns.load_dataset('tips')
# tips.head()

In [14]:
# tips.describe(include = 'all')

### Reading from url

You can also load datasets from urls where a `.csv` (or other) file live.  Github is one example of this.  Note that you want to be sure to use the *raw* version of the file.  For example, a github user `dsnair` has shared datasets from the book [Introduction to Statistical Learning]() at the link below:


https://github.com/dsnair/ISLR/tree/master/data/csv

read in the Auto dataset below.

In [15]:
# get url to raw data
auto_url = 'https://raw.githubusercontent.com/dsnair/ISLR/master/data/csv/Auto.csv'

In [16]:
# pass to read_csv
auto = pd.read_csv(auto_url)

In [17]:
#auto.describe?

In [18]:
# look at the first few rows
auto.head(2)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320


In [19]:
# high level information
auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    int64  
 4   weight        392 non-null    int64  
 5   acceleration  392 non-null    float64
 6   year          392 non-null    int64  
 7   origin        392 non-null    int64  
 8   name          392 non-null    object 
dtypes: float64(3), int64(5), object(1)
memory usage: 27.7+ KB


#### Problems

1. Read in the `diamonds.csv` file from the `data` folder, and create a DataFrame named `diamonds`.

In [20]:
diamonds_path = 'data/diamonds.csv'


2. How many diamonds are greater than .5 carat in the data?

3. What is the highest priced diamond in the data?

4. Read the data from the `caravan.csv` file in located [here](https://github.com/dsnair/ISLR/blob/master/data/csv/Caravan.csv).  Assign this to a variable `caravan`.

5. How many `Yes`'s are in the `Purchase` column of the `caravan` data?  No's?

### Missing Values

Missing values are a common problem in data, whether this is because they are truly missing or there is confusion between the data encoding and the methods you read the data in using.

In [21]:
# re-examine ufo info


In [22]:
# one-liner to count missing values


In [23]:
# drop missing values


In [24]:
# fill missing values


In [25]:
# replace missing values with most common value


In [26]:
# be careful with comparisons!!!


#### Problem

1. Read in the dataset `churn_missing.csv` in the data folder, assign to a variable `churn` below.

2. Are there any missing values?  What columns are they in and how many are there?

3. What do you think we should do about these?  Drop, replace, impute?

#### `groupby`

Often, you are faced with a dataset that you are interested in summaries within groups based on a condition.  The simplest condition is that of a unique value in a single column.  Using `.groupby` you can split your data into unique groups and summarize the results.  

**NOTE**: After splitting you need to summarize!

![](https://www.oreilly.com/api/v2/epubs/9781783985128/files/graphics/5128OS_09_01.jpg)

In [27]:
# sample data
df = pd.DataFrame(
    {
        "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
        "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
        "C": np.random.randn(8),
        "D": np.random.randn(8),
    }
)
df

,A,B,C,D
0,foo,one,0.014738,-0.549100
1,bar,one,-1.515979,-0.905696
2,foo,two,-1.034411,-0.157872
3,bar,three,0.392896,-1.189066
4,foo,two,-0.260686,-0.671384
5,bar,two,-1.633476,0.943162
6,foo,one,-1.707952,-0.037748
7,foo,three,-0.709300,0.772273


In [28]:
# foo vs. bar
df.groupby('A')['C'].mean()

A
bar   -0.918853
foo   -0.739522
Name: C, dtype: float64

In [29]:
# one two or three?
df.groupby('B').mean(numeric_only=True)

,C,D
B,,
one,-1.069731,-0.497514
three,-0.158202,-0.208397
two,-0.976191,0.037969


In [30]:
# A and B
df.groupby(['A', 'B']).mean()

C         D
A   B                        
bar one   -1.515979 -0.905696
    three  0.392896 -1.189066
    two   -1.633476  0.943162
foo one   -0.846607 -0.293424
    three -0.709300  0.772273
    two   -0.647549 -0.414628

In [31]:
# working with multi-index
df.groupby(['A', 'B'], as_index=False).mean()

,A,B,C,D
0,bar,one,-1.515979,-0.905696
1,bar,three,0.392896,-1.189066
2,bar,two,-1.633476,0.943162
3,foo,one,-0.846607,-0.293424
4,foo,three,-0.709300,0.772273
5,foo,two,-0.647549,-0.414628


In [32]:
# age less than 40 survival rate
titanic = sns.load_dataset('titanic')
titanic.groupby(titanic['age'] < 40)[['survived']].mean()

,survived
age,
False,0.332353
True,0.415608


##### Problems

In [33]:
tips = sns.load_dataset('tips')

In [34]:
tips.head(2)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3


1. Average tip for smokers vs. non-smokers.

2. Average bill by day and time.

3. What is another question `groupby` can help us answer here?

4. What does the `as_index` argument do?  Demonstrate an example.

#### `datetime`

A special type of data for pandas are entities that can be considered as dates.  We can create a special datatype for these using `pd.to_datetime`, and access the functions of the `datetime` module as a result.

In [35]:
# read in the AAPL data
aapl = pd.read_csv('data/AAPL.csv')
aapl.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2005-04-25,5.212857,5.288571,5.158571,5.282857,3.522625,186615100
1,2005-04-26,5.254286,5.358572,5.160000,5.170000,3.447372,202626900
2,2005-04-27,5.127143,5.194286,5.072857,5.135714,3.424510,153472200
3,2005-04-28,5.184286,5.191429,5.034286,5.077143,3.385454,143776500
4,2005-04-29,5.164286,5.175714,5.031428,5.151429,3.434988,167907600


In [36]:
# convert to datetime
aapl['Date'] = pd.to_datetime(aapl['Date'])

In [37]:
# extract the month
aapl['month'] = aapl['Date'].dt.month

In [38]:
# extract the day
aapl['day'] = aapl['Date'].dt.day

In [39]:
# set date to be index of data
aapl.set_index('Date', inplace = True)

In [40]:
# sort the index
aapl.sort_index(inplace = True)

In [41]:
# select 2019
aapl.loc['2019']

,Open,High,Low,Close,Adj Close,Volume,month,day
Date,,,,,,,,
2019-01-02,154.889999,158.850006,154.229996,157.919998,157.245605,37039700,1,2
2019-01-03,143.979996,145.720001,142.000000,142.190002,141.582779,91244100,1,3
2019-01-04,144.529999,148.550003,143.800003,148.259995,147.626846,58607100,1,4
2019-01-07,148.699997,148.830002,145.899994,147.929993,147.298264,54777800,1,7
2019-01-08,149.559998,151.820007,148.520004,150.750000,150.106216,41025300,1,8
...,...,...,...,...,...,...,...,...
2019-04-16,199.460007,201.369995,198.559998,199.250000,199.250000,25696400,4,16
2019-04-17,199.539993,203.380005,198.610001,203.130005,203.130005,28906800,4,17
2019-04-18,203.119995,204.149994,202.520004,203.860001,203.860001,24195800,4,18


In [42]:
# read back in using parse_dates = True and index_col = 0
aapl = pd.read_csv('data/AAPL.csv', parse_dates=True, index_col=0)
aapl.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3523 entries, 2005-04-25 to 2019-04-23
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3523 non-null   float64
 1   High       3523 non-null   float64
 2   Low        3523 non-null   float64
 3   Close      3523 non-null   float64
 4   Adj Close  3523 non-null   float64
 5   Volume     3523 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 192.7 KB


In [43]:
from datetime import datetime

In [44]:
# what time is it?
then = datetime.now()
then

datetime.datetime(2023, 10, 4, 21, 45, 49, 915930)

In [45]:
# how much time has passed?
datetime.now() - then

datetime.timedelta(microseconds=6784)

#### More with timestamps

- Date times: A specific date and time with timezone support. Similar to datetime.datetime from the standard library.

- Time deltas: An absolute time duration. Similar to datetime.timedelta from the standard library.

- Time spans: A span of time defined by a point in time and its associated frequency.

- Date offsets: A relative time duration that respects calendar arithmetic.

In [46]:
# create a pd.Timedelta
pd.Timedelta(5, 'D')

Timedelta('5 days 00:00:00')

In [47]:
# shift a date by 3 months
then + pd.Timedelta(36, 'W')

datetime.datetime(2024, 6, 12, 21, 45, 49, 915930)

##### Problems

1. Return to the ufo data and convert the Time column to a datetime object.

2. Set the Time column as the index column of the data.

3. Sort it

4. Create a new dataframe with ufo sightings since January 1, 1999